In [1]:
import os
import pandas as pd
import numpy as np


In [2]:
annotations = os.listdir('./annotations')
#print([x.split('.')[0] for x in annotations])
images = os.listdir('./images')
print(len(images))
df = pd.DataFrame(0, index=np.arange(1,len(images)+1), columns=[x.split('.')[0] for x in annotations])
print(df.shape)
for tag in annotations:
    with open(f'./annotations/{tag}') as f:
        lines = f.readlines()
        for line in lines:
           imgNumber = line.split("\n")[0]
           df[tag.split('.')[0]][int(imgNumber)] = 1

20000
(20000, 14)


In [4]:
import torch
import torch.optim as optim
import torch.utils.data
import torch.backends.cudnn as cudnn
import torchvision
from torchvision import transforms, datasets
import torch.nn as nn
import torch.nn.functional as F

In [5]:
from PIL import Image
from torchvision.io import read_image
from torch.utils.data import Dataset

In [6]:
DATA_DIR = 'images'

In [7]:
train_transform = transforms.Compose([transforms.Grayscale()])
test_transform = transforms.Compose([transforms.Grayscale()])

In [17]:
#this code is an edited version of the code found at https://pytorch.org/tutorials/beginner/basics/data_tutorial.html

class CustomImageDataset(Dataset):
    def __init__(self, annotations_df, img_dir, transform=None, target_transform=None):
        self.img_labels = annotations_df
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, "im"+str(idx+1)+".jpg")
        image = read_image(img_path)
        label = torch.from_numpy(np.array(self.img_labels.iloc[idx]))
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label


In [18]:
customImageDataset = CustomImageDataset(df, DATA_DIR, transform=train_transform)

In [19]:
train_loader = torch.utils.data.DataLoader(dataset=customImageDataset, batch_size=1, shuffle=True)

In [21]:
for batch_num, (data, target) in enumerate(train_loader):
    print(data)

tensor([[[[ 50,  51,  51,  ...,  44,  44,  44],
          [ 51,  51,  51,  ...,  45,  45,  45],
          [ 51,  51,  51,  ...,  46,  46,  47],
          ...,
          [128, 140, 109,  ..., 114, 127, 122],
          [120,  86, 104,  ..., 115, 117, 125],
          [ 87,  75, 110,  ..., 128, 142, 137]]]], dtype=torch.uint8)
tensor([[[[132, 130, 137,  ...,  87,  93,  90],
          [140, 135, 132,  ..., 132, 107, 144],
          [143, 141, 132,  ..., 194, 124, 131],
          ...,
          [142, 148, 153,  ..., 124, 142, 135],
          [133, 138, 152,  ..., 117, 125, 129],
          [132, 135, 145,  ..., 131, 122, 120]]]], dtype=torch.uint8)
tensor([[[[222, 228, 212,  ..., 115, 131,  97],
          [206, 234, 225,  ...,  54,  78, 109],
          [218, 209, 224,  ...,  12,  13,  28],
          ...,
          [102,  87, 106,  ..., 128, 112, 102],
          [100,  86,  91,  ..., 127, 139, 143],
          [101,  81,  88,  ...,  95,  80, 116]]]], dtype=torch.uint8)
tensor([[[[ 50,  42,  45,

KeyboardInterrupt: 